## Use Case 1
* Extract file properties from all parts, put them into an excel file
* Modify the excel file
* Extract data from modified excel file, use this to modify data in part files.
* Generate the modified BOM (done within Solidworks and not using python)

## Use Case 2
* Changing property values in multiple part files (currently supports same values only)
* Eg. 10 part files need their project names changed

In [4]:
list2 = [1,2,3]
a = str(list2)
print(a)

print(type(a))


[1, 2, 3]
<class 'str'>


In [5]:
import pandas as pd

# Sample DataFrames for demonstration
# Ensure initial_df and final_df have the same index and columns
initial_df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
}, index=['row1', 'row2', 'row3'])

print(initial_df)

final_df = pd.DataFrame({
    'A': [1, 2, 4],
    'B': [4, 0, 6],
    'C': [7, 8, 10]
}, index=['row1', 'row2', 'row3'])

print(final_df)

# Create a Boolean mask where values differ between initial_df and final_df
mask = (initial_df != final_df).any(axis=1)

# Use the mask to filter initial_df for rows with changes
changed_rows_df = final_df[mask]


changes = final_df[~final_df.eq(initial_df).all(axis=1)]


# Display the result
print("Subset of initial_df with rows where values have changed:")

print(changes)
# print(changed_rows_df)

      A  B  C
row1  1  4  7
row2  2  5  8
row3  3  6  9
      A  B   C
row1  1  4   7
row2  2  0   8
row3  4  6  10
Subset of initial_df with rows where values have changed:
      A  B   C
row2  2  0   8
row3  4  6  10


In [2]:
import pySldWrap.sw_tools as sw_tools
import importlib
import os
from pathlib import Path
import time 

In [5]:

# !Solidworks must be open before running the code below: 

#reloads sw_tools module so changes in .py file actually work without restarting the kernel when running the cell blocks below
#reload causes all variables to be lost; do not do it in other cells besides this block
importlib.reload(sw_tools)
sw_tools.connect_sw("2024")  # open connection and pass Solidworks version

### Retrieve file properties for a **single part**

In [95]:
'''
Code for a single part
'''
# part_path = './Test_files\LS3.SLDPRT'
# model = sw_tools.open_part(part_path)  # open the model, link is returned
# custom_properties = sw_tools.get_custom_file_properties(part_path)

# a = sw_tools.export_custom_file_properties(custom_properties)

'\nCode for a single part\n'

### Retrieve file properties for **multiple parts**

In [17]:
importlib.reload(sw_tools)
sw_tools.connect_sw("2024")

directory = "./Test_files"
part_path = ''

# start_time = time.perf_counter()

for path, folders, files in os.walk(directory):
    for filename in files:
        # check for part files and assemblies
        try:
            if filename.endswith(('.SLDPRT','.SLDASM')):
                part_path = os.path.join(directory, filename)
                model = sw_tools.open_part(part_path)
                custom_properties = sw_tools.get_custom_file_properties(model)
                sw_tools.export_custom_file_properties(custom_properties)
                #close files to reduce memory taken up; essential for reducing time taken for whole process
                sw_tools.close(part_path.split('\\')[-1])
        except:
            continue

# end_time = time.perf_counter()
# execution_time = end_time - start_time
# print(f"Execution time: {execution_time:.2f} seconds")

### Modify values in solidworks part file/assembly using excel file

In [4]:

# !need to make this work with actual BOM
# !convert to exe?

importlib.reload(sw_tools)
sw_tools.connect_sw("2024")

# user will input absolute location? of excel file
# user will input absoute location? of part files and assemblies
import pandas as pd
import datetime

directory = "./Test_files"
# part_path = ''

def modify_file_properties_from_excel(directory,filename):
    '''
    Reads data from an existing BOM in an excel file
    Reads names and values of file properties in excel file
    Checks these against names and values in part/assembly files
    Overwrites them
    '''
    # *create a for loop that does this for every name in the title column:
    # *read the title and associate it with a name of a part file or assembly; if title contains assembly then its an assembly; if not its not
    # *open the specified file or assembly
    # *get the file properties in that part file/assembly
    # *replace the file properties in the part file/assembly with those in the excel file

    # Generate a new log file name
    logfile_count = 1
    while os.path.exists(f"Solidworks_Log_File{logfile_count}.txt"):
        logfile_count += 1
    logfile_name = f"Solidworks_Log_File{logfile_count}.txt"

    excel_df = pd.read_excel(filename)
    column_list = list(excel_df.loc[:,"Title"])
    #accesses the first excel row to retrieve names of rows
    excel_df_2 = excel_df.set_index(excel_df.columns[0])

    skip_columns = {'Enterprise Part No.', 'Title', 'V_Name', 'Revision', 'Creation Date', 'DrawnDate', 'Material', 'CheckedDate', 'EngAppDate', 'MfgAppDate', 'QAAppDate', 'Remarks'}
    new_excel_df = excel_df_2.drop(columns=skip_columns, errors='ignore')

    for i in range(excel_df.shape[0]):
        #checking if its a part file or assembly file
        if ("Assembly" or "Assem") in column_list[i]:
            filename = column_list[i] + '.SLDASM'
        else:
            filename = column_list[i] + '.SLDPRT'

        property_value_list = list(new_excel_df.loc[column_list[i],:])

        #open part/assembly and get custom file properties
        try:
            part_path = os.path.join(directory, filename)
            print(part_path)
            model = sw_tools.open_part(part_path)
            sw_tools.set_file_properties(model,property_value_list)
            sw_tools.close(part_path.split('\\')[-1])
        
        # Create log file for error handling
        except Exception as e:
            error_timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f'Error processing {filename}: {e}. See {logfile_name} for details.')
            with open(logfile_name, 'a') as f:
                f.write(f"{error_timestamp} - {part_path} - Error: {e}\n")  

modify_file_properties_from_excel(directory,'custom_properties.xlsx')


./Test_files\LS1.SLDPRT
./Test_files\LS2.SLDPRT
./Test_files\LS3.SLDPRT
./Test_files\LS4a.SLDPRT
./Test_files\LS4b.SLDPRT
./Test_files\LS5.SLDPRT


In [2]:
import pySldWrap.sw_tools as sw_tools
import importlib
import os
import time 
from pathlib import Path

import tkinter as tk
from tkinter import filedialog
import os
import pandas as pd
import win32com.client
import win32gui

from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

import datetime
import numpy as np

In [3]:
def prompt_user_for_path(path_name,path_type):
    '''
    Gets filename for some processing, returns file_path for additional processing
    '''
    root = tk.Tk()
    root.withdraw()

    script_dir = os.path.dirname(os.getcwd())

    #get file path or directory
    if path_type == 'File':
        #get file path
        file_path = filedialog.askopenfilename(
            title=f"{path_name}", 
            initialdir=script_dir 
        )
    if path_type == 'Directory':
        file_path = filedialog.askdirectory(
            title=f"{path_name}", 
            initialdir=script_dir 
        )

    # Bring the file dialog window to the front
    root.deiconify()  # Make the root window visible (it's already hidden)
    root.focus_force()  # Force the root window to get focus
    root.after(1, lambda: root.withdraw())  # Hide the root window after a short delay

    return file_path

In [5]:
import pandas as pd
import datetime

importlib.reload(sw_tools)
sw_tools.connect_sw("2024")

directory = prompt_user_for_path('Select your assembly/part file directory','Directory')
# part_path = ''

def modify_sw_file_properties(directory, df_of_modified_files):
    '''
    Reads from a dataframe containing names of partfiles/assemblies whose file properties have been modified
    Iterates through these files based on a user provided directory
    

    Reads names and values of file properties in excel file
    Checks these against names and values in part/assembly files
    Overwrites them
    '''
    # *create a for loop that does this for every name in the title column:
    # *read the title and associate it with a name of a part file or assembly; if title contains assembly then its an assembly; if not its not
    # *open the specified file or assembly
    # *get the file properties in that part file/assembly
    # *replace the file properties in the part file/assembly with those in the excel file

    # Generate a new log file name
    logfile_count = 1
    while os.path.exists(f"Solidworks_Log_File{logfile_count}.txt"):
        logfile_count += 1
    logfile_name = f"Solidworks_Log_File{logfile_count}.txt"

    column_list = list(df_of_modified_files.loc[:,"Title"])
    new_column_list = [string.replace('\n', '') for string in column_list]

    #accesses the first excel row to retrieve names of rows
    # excel_df_2 = df_of_modified_files.set_index(df_of_modified_files.columns[1])



    # print('excel df:',excel_df_2)

    skip_columns = {'S/N','Enterprise Part No.', 'Description', 'V_Name', 'Revision', 'Creation Date', 'DrawnDate', 'Material', 'CheckedDate', 'EngAppDate', 'MfgAppDate', 'QAAppDate', 'Remarks', 'DrawnBy', 'CheckedBy', 'EngApproval', 'MfgApproval', 'QAApproval'}


    # new_excel_df = excel_df_2.drop(columns=skip_columns, errors='ignore')

    new_excel_df = df_of_modified_files.drop(columns=skip_columns, errors='ignore')

    print('excel df:',new_excel_df)


    for i in range(df_of_modified_files.shape[0]):
        #checking if its a part file or assembly file

        if "Assembly" in str(new_column_list[i]) or "Assem" in str(new_column_list[i]):
            filename = new_column_list[i] + '.SLDASM'
        else:
            filename = new_column_list[i] + '.SLDPRT'

        property_value_list = list(new_excel_df.loc[column_list[i],:])

        property_value_list = [str(value) if pd.notna(value) else '--' for value in list(new_excel_df.loc[column_list[i],:])]

        print('proplist:',property_value_list)

        #open part/assembly and get custom file properties
        try:
            part_path = os.path.join(directory, filename)
            print(part_path)
            if "Assembly" in filename or "Assem" in filename:
                model = sw_tools.open_assembly(part_path)
            else:
                model = sw_tools.open_part(part_path)
            sw_tools.set_file_properties(model,property_value_list)
            sw_tools.close(part_path.split('\\')[-1])
        
        # Create log file for error handling
        except Exception as e:
            error_timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f'Error processing {filename}: {e}. See {logfile_name} for details.')
            with open(logfile_name, 'a') as f:
                f.write(f"{error_timestamp} - {part_path} - Error: {e}\n")  